In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_Cancer"
cohort = "GSE148346"

# Input paths
in_trait_dir = "../../input/GEO/Liver_Cancer"
in_cohort_dir = "../../input/GEO/Liver_Cancer/GSE148346"

# Output paths
out_data_file = "../../output/preprocess/Liver_Cancer/GSE148346.csv"
out_gene_data_file = "../../output/preprocess/Liver_Cancer/gene_data/GSE148346.csv"
out_clinical_data_file = "../../output/preprocess/Liver_Cancer/clinical_data/GSE148346.csv"
json_path = "../../output/preprocess/Liver_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# From the background information, this is a study on alopecia areata (AA) with molecular response data
# This indicates gene expression data is likely available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary

# 2.1 For the trait (Liver Cancer)
# Looking at the background information and sample characteristics, this dataset doesn't appear to be about liver cancer
# It's a study on alopecia areata (AA)
# Key 3 shows 'tissue disease state: LS', 'tissue disease state: NL' which relates to lesional (LS) and non-lesional (NL) skin
trait_row = 3  # Using tissue disease state as our trait indicator

# For age - Not available in the sample characteristics dictionary
age_row = None

# For gender - Not available in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert trait values to binary (1 for lesional, 0 for non-lesional)"""
    if pd.isna(value):
        return None
    value_str = str(value).strip()
    if ":" in value_str:
        value_str = value_str.split(":", 1)[1].strip()
    
    if value_str.upper() == "LS":
        return 1  # Lesional skin
    elif value_str.upper() == "NL":
        return 0  # Non-lesional skin
    else:
        return None

def convert_age(value):
    """Convert age values to continuous (Not used in this dataset)"""
    return None  # Placeholder since age data is not available

def convert_gender(value):
    """Convert gender values to binary (Not used in this dataset)"""
    return None  # Placeholder since gender data is not available

# 3. Save Metadata
# Check if trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # If trait data is available, extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_data,  # Assuming clinical_data is available from previous step
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# Check if the dataset contains gene expression data based on previous assessment
if not is_gene_available:
    print("This dataset does not contain gene expression data (only miRNA data).")
    print("Skipping gene expression data extraction.")
else:
    # Get the matrix file directly rather than using geo_get_relevant_filepaths
    files = os.listdir(in_cohort_dir)
    if len(files) > 0:
        matrix_file = os.path.join(in_cohort_dir, files[0])
        print(f"Matrix file found: {matrix_file}")
        
        try:
            # Extract gene data
            gene_data = get_genetic_data(matrix_file)
            print(f"Gene data shape: {gene_data.shape}")
            
            # Print the first 20 gene/probe identifiers
            print("First 20 gene/probe identifiers:")
            print(gene_data.index[:20])
        except Exception as e:
            print(f"Error extracting gene data: {e}")
    else:
        print("No files found in the input directory.")


### Step 4: Gene Identifier Review

In [ ]:
# These identifiers ("1007_s_at", "1053_at", etc.) are Affymetrix probe IDs from a microarray platform,
# not standard human gene symbols. They need to be mapped to official gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())


### Step 6: Gene Identifier Mapping

In [ ]:
# Identify the columns for mapping
# From the gene annotation preview, I can see that:
# - 'ID' column contains the probe identifiers (e.g., '1007_s_at')
# - 'Gene Symbol' column contains the gene symbols (e.g., 'DDR1 /// MIR4640')

# 1. Get the mapping between probe IDs and gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# Print mapping dataframe info
print("\nGene mapping dataframe info:")
print(f"Shape: {mapping_df.shape}")
print(mapping_df.head())

# 2. Apply the mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_df)

# 3. Normalize gene symbols (handle synonyms and merge duplicate symbols)
gene_data = normalize_gene_symbols_in_index(gene_data)

# Print gene expression data info
print("\nGene expression data info after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")
print(f"First 5 sample IDs: {gene_data.columns[:5].tolist()}")

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# Load the clinical data from the file we saved in step 2
clinical_data_file = out_clinical_data_file
if os.path.exists(clinical_data_file):
    selected_clinical_df = pd.read_csv(clinical_data_file)
    print(f"Loaded clinical data from {clinical_data_file}")
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())
else:
    print(f"Clinical data file {clinical_data_file} not found. Re-extracting clinical features...")
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    print("Re-extracted clinical data preview:")
    print(preview_df(selected_clinical_df))

# 1. Normalize gene symbols in the index
print("\nNormalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print("First 10 gene identifiers after normalization:")
print(normalized_gene_data.index[:10].tolist())

# Save the normalized gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("\nLinking clinical and genetic data...")
# Since we read clinical data with a standard index (0, 1, 2...), need to transpose before linking
if 'Liver_Cancer' in selected_clinical_df.columns:
    selected_clinical_df.set_index('Liver_Cancer', inplace=True)
    selected_clinical_df = selected_clinical_df.T
else:
    # Transpose to get samples as rows and trait as column
    selected_clinical_df = selected_clinical_df.T
    selected_clinical_df.columns = [trait]

linked_data = pd.concat([selected_clinical_df, normalized_gene_data.T], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
print(linked_data.iloc[:5, :5])

# 3. Handle missing values in the linked data
print("\nHandling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine if the trait and demographic features are biased
print("\nChecking for bias in trait and demographic features...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct final quality validation and save relevant information
print("\nConducting final quality validation...")
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = True  # We've confirmed trait data is available in previous steps

note = "This dataset contains gene expression data from skin biopsies of patients with alopecia areata, comparing lesional and non-lesional samples. The dataset is actually about alopecia areata, not liver cancer."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved as dataset is not usable for the current trait study.")